##### Import Necessary Packages

In [6]:
import numpy as np
import pandas as pd
import os

In [7]:
# --- Configuration ---
RESULTS_DIR = "../../temp_results/"
FOLDERS = ["AM", "MOEAAM", "MOEAAM_RS"]
DATASETS = ["mushrooms", "tictactoe"]
OBJECTIVES = ["spec_sens", "conf_simp", "conf_spec", "conf_sens"]

# Metrics you want in the columns
TARGET_METRICS = [
    "accuracy", "f1_score", "recall", "precision", "specificity", 
    "nb_rules", "term_rule_ratio", "hypervolume"
]

In [ ]:
def process_results():
    # Dictionary to store the final dataframe for each dataset
    dataset_results = {}

    for dataset in DATASETS:
        rows = []
        
        for folder in FOLDERS:
            for obj in OBJECTIVES:
                # Construct file path: ../../temp_results/AM/mushrooms_spec_sens.csv
                file_path = os.path.join(RESULTS_DIR, folder, f"{dataset}_{obj}.csv")
                
                if not os.path.exists(file_path):
                    continue
                
                try:
                    # Load data
                    df = pd.read_csv(file_path)
                    
                    # 1. Filter for TEST results only
                    # We assume we want to aggregate the 'test' split rows
                    df_test = df[df['split'] == 'test'].copy()
                    
                    if df_test.empty:
                        continue

                    # 2. Normalize 0-1 scales to 0-100 if necessary
                    # We check if the average accuracy is <= 1.0. If so, multiply by 100.
                    # This fixes the discrepancy between AM (0.79) and MOEA (85.4)
                    if 'accuracy' in df_test.columns and df_test['accuracy'].mean() <= 1.0:
                        cols_to_scale = ["accuracy", "f1_score", "recall", "precision", "specificity"]
                        for col in cols_to_scale:
                            if col in df_test.columns:
                                df_test[col] = df_test[col] * 100

                    # 3. Aggregate (Average over runs and folds)
                    # We take the mean of all numeric columns
                    numeric_cols = df_test.select_dtypes(include='number').columns
                    means = df_test[numeric_cols].mean()

                    # 4. Construct the row
                    row_data = {
                        "algo_obj": f"{folder}_{obj}",  # This will be the index
                    }
                    
                    # Add metrics to row
                    for metric in TARGET_METRICS:
                        if metric in means:
                            row_data[metric] = means[metric]
                        else:
                            row_data[metric] = None # e.g. hypervolume for AM

                    rows.append(row_data)

                except Exception as e:
                    print(f"Error processing {file_path}: {e}")

        # Create DataFrame for this dataset if we found data
        if rows:
            res_df = pd.DataFrame(rows)
            res_df.set_index("algo_obj", inplace=True)
            dataset_results[dataset] = res_df
        else:
            dataset_results[dataset] = pd.DataFrame()

    return dataset_results

# --- Execution ---
dfs = process_results()

dfs["mushrooms"]

,accuracy_train,accuracy_test,f1_score_train,f1_score_test,recall_train,recall_test,precision_train,precision_test,specificity_train,specificity_test,nb_rules_train,nb_rules_test,term_rule_ratio_train,term_rule_ratio_test,time_train,time_test,hypervolume_train,hypervolume_test
algo_obj,,,,,,,,,,,,,,,,,,
AM_spec_sens,90.983910,90.963494,90.908794,90.881227,90.983910,90.963494,91.693957,91.714081,90.572651,90.560751,86.8,86.8,1.026363,1.026363,60.403920,60.403920,NaN,NaN
AM_conf_simp,99.326060,99.396893,99.325686,99.396634,99.326060,99.396893,99.340287,99.407394,100.000000,100.000000,126.4,126.4,1.006625,1.006625,60.332786,60.332786,NaN,NaN
AM_conf_spec,99.784592,99.803047,99.784572,99.803027,99.784592,99.803047,99.785525,99.804036,99.964349,99.952494,130.8,130.8,1.257696,1.257696,60.363020,60.363020,NaN,NaN
AM_conf_sens,91.294199,91.199363,91.182578,91.089205,91.294199,91.199363,92.618117,92.530590,100.000000,100.000000,57.2,57.2,1.034499,1.034499,63.493467,63.493467,NaN,NaN
MOEAAM_spec_sens,71.584291,71.565297,69.587923,69.543092,71.584291,71.565297,80.206051,80.321066,47.095409,46.979814,5.2,5.2,1.033333,1.033333,60.514111,60.514111,0.741916,0.741916
MOEAAM_conf_simp,99.704578,99.704555,99.704543,99.704514,99.704578,99.704555,99.706272,99.706530,100.000000,100.000000,17.0,17.0,1.000000,1.000000,60.367460,60.367460,1.000000,1.000000
MOEAAM_conf_spec,99.947684,99.938462,99.947683,99.938458,99.947684,99.938462,99.947775,99.938622,100.000000,100.000000,179.6,179.6,2.061997,2.061997,60.326364,60.326364,1.000000,1.000000
MOEAAM_conf_sens,73.950794,73.718909,72.313895,72.047151,73.950794,73.718909,81.719571,81.568479,51.342215,50.903008,4.6,4.6,1.000000,1.000000,60.548880,60.548880,0.742682,0.742682
MOEAAM_RS_spec_sens,84.185438,84.331626,83.469486,83.661735,84.185438,84.331626,88.693506,88.732384,69.800683,70.036773,46.6,46.6,1.114293,1.114293,60.934488,60.934488,0.660657,0.660657
